# Toxic Language Detection on Jigsaw Dataset

In this notebook we're training a multi-label text classifier. 

## Toy Example

In [1]:
import pandas as pd

from simpletransformers.classification import MultiLabelClassificationModel

In [ ]:
train_data = [
    ["Pizza and pasta are Italian food", [1, 1, 0, 0, 0, 0]],
    ["Before start cooking find a good recipe", [1, 0, 0, 0, 0, 0]],
    ["Cooking is one of my hobbies", [0, 1, 0, 0, 0, 0]],
    ["I like football", [0, 0, 1, 1, 0, 0]],
    ["I hate tennis", [0, 0, 1, 0, 0, 0]],
    ["This year the Olympic Games are held in Tokyo", [0, 0, 0, 1, 0, 0]],
    ["Natural Language Processing deals with talking machines", [0, 0, 0, 0, 1, 1]],
    ["Textual entailment and semantic similarity are NLP tasks", [0, 0, 0, 0, 1, 0]],
    ["NLU stands for natural language understanding", [0, 0, 0, 0, 0, 1]],
]

train_df = pd.DataFrame(train_data, columns=["text", "labels"])

eval_data = [
    ["Cooking is one of my hobbies", [0, 1, 0, 0, 0, 0]],
    ["I hate tennis", [0, 0, 1, 0, 0, 0]],
    ["Natural Language Processing deals with talking machines", [0, 0, 0, 0, 1, 1]],
]

eval_df = pd.DataFrame(eval_data, columns=["text", "labels"])

In [ ]:
# configuration
args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",
    "fp16": False,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 32,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "logging_steps": 50,
    "save_steps": 2000,
    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": False,
    # "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
}

In [ ]:
model = MultiLabelClassificationModel(
    "roberta", "roberta-base", num_labels=6, use_cuda=False, args=args
)

# Train the model
model.train_model(train_df)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)

In [ ]:
print(model_outputs)

In [ ]:
predictions, raw_outputs = model.predict(["This class is about natural language."])

print(predictions)
print(raw_outputs)

## Training a Text Classifier to Detect Toxic Language

In [2]:
train = pd.read_csv("../datasets/toxic_language/train.csv")
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [3]:
# encode the labels in a new column
labels = list(
    map(
        list,
        zip(
            train["toxic"],
            train["severe_toxic"],
            train["obscene"],
            train["threat"],
            train["insult"],
            train["identity_hate"],
        ),
    )
)

train["labels"] = labels

# rename and remove other columns
train = train.rename(columns={"comment_text": "text"})
train = train.drop(
    "id toxic severe_toxic obscene threat insult identity_hate".split(), axis=1
)

train.head(10)

,text,labels
0,Explanation\nWhy the edits made under my usern...,"[0, 0, 0, 0, 0, 0]"
1,D'aww! He matches this background colour I'm s...,"[0, 0, 0, 0, 0, 0]"
2,"Hey man, I'm really not trying to edit war. It...","[0, 0, 0, 0, 0, 0]"
3,"""\nMore\nI can't make any real suggestions on ...","[0, 0, 0, 0, 0, 0]"
4,"You, sir, are my hero. Any chance you remember...","[0, 0, 0, 0, 0, 0]"
5,"""\n\nCongratulations from me as well, use the ...","[0, 0, 0, 0, 0, 0]"
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,"[1, 1, 1, 0, 1, 0]"
7,Your vandalism to the Matt Shirvington article...,"[0, 0, 0, 0, 0, 0]"
8,Sorry if the word 'nonsense' was offensive to ...,"[0, 0, 0, 0, 0, 0]"
9,alignment on this subject and which are contra...,"[0, 0, 0, 0, 0, 0]"


In [4]:
# configuration
args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",
    "fp16": False,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 64,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 10,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "logging_steps": 50,
    "save_steps": 2000,
    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": False,
    # "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
    "wandb_project": "nlp-exercises",
}

In [ ]:
model = MultiLabelClassificationModel(
    "roberta", "roberta-base", num_labels=6, args=args
)

# Train the model
model.train_model(train)

Features loaded from cache at cache_dir/cached_train_roberta_128_6_159571


Running loss: 0.686984

/opt/conda/envs/nlp/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.117438

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)

In [ ]:
print(model_outputs[:10])

In [ ]:
predictions, raw_outputs = model.predict(["You fucking idiot!!"])

print(predictions)
print(raw_outputs)

## A Quick Demo

In [4]:
def load_model(
    model_architecture: str,
    directory: str = "outputs/",
    num_labels: int = 6,
    use_cuda: bool = False,
    **kwargs
):
    """Loads a pre-trained model"""
    model = MultiLabelClassificationModel(
        model_architecture, directory, num_labels=6, use_cuda=use_cuda, args=kwargs
    )
    return model

In [5]:
model = load_model("roberta")

In [6]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def detect_toxic_lang(line, text):
    """Prints predictions of a Text Classifier"""
    predictions, raw_outputs = model.predict([text])
    return predictions[0]

In [7]:
%%detect_toxic_lang
Hello, sir. How is your day?

Converting to features started. Cache is not used.


[0, 1, 1, 0, 0, 0]

In [8]:
%%detect_toxic_lang
Only nigga that I trust is me

Converting to features started. Cache is not used.


[0, 1, 1, 0, 0, 0]